In [2]:
import pandas as pd
import requests
import json
import html
import datetime
import csv
import re


In [3]:
companies_df = pd.read_csv('../lever_companies.csv')
companies_df


,Company,Token
0,Anduril Industries,anduril
1,Angle Health,AngleHealth
2,Anyscale,anyscale
3,Ascend.io,ascend.io
4,Asics,?asics?
...,...,...
57,Up&Up,?Up&Up?
58,Veeva,veeva
59,Xage Security,xage-security
60,YugabyteDB,yugabyte


In [ ]:
companies_dict = {}
for _ , row in companies_df.iterrows():
  companies_dict[row['Token']] = row['Company']


companies_dict


{'AngleHealth': 'Angle Health',
 'torch': 'Torch',
 'tegus': 'Tegus',
 'secureframe': 'secureframe',
 'bluebottlecoffee': 'Blue Bottle Coffee',
 'zoox': 'Zoox',
 'front': 'Front',
 'eargo': 'Eargo',
 'cohere': 'Cohere',
 'anduril': 'Anduril Industries',
 'hihello': 'HiHello',
 'plaid': 'Plaid',
 'leolabs-2': 'LeoLabs',
 'veeva': 'Veeva',
 'MachinaLabs': 'Machina Labs',
 'terminus': 'Terminus',
 'credinvestments': 'CRED Investments',
 'axoni': 'Axoni',
 'observeinc': 'Observe',
 'roboflow': 'Roboflow',
 'sporty': 'Sporty Group',
 'ottertune': 'OtterTune',
 'hive': 'Hive',
 'orum': 'Orum',
 'boomandbucket': 'Boom & Bucket',
 'ascend.io': 'Ascend.io',
 'aurorasolar': 'Aurora Solar',
 'getlabs': 'Getlabs',
 'Pomelo': 'Pomelo',
 'OpenSea': 'OpenSea',
 'logrocket': 'LogRocket',
 'netflix': 'Netflix',
 'talyn': 'Talyn Air',
 'nava': 'Nava',
 'nexient': 'Nexient',
 'goforward': 'Forward',
 'gopuff': 'Gopuff',
 'anyscale': 'Anyscale',
 'automox': 'Automox',
 'Sherpa': 'Sherpa',
 'learnwithjuni'

In [ ]:
tokens = set(companies_df.Token)
companies_clean = companies_dict
companies_bad = {}

In [ ]:
len(companies_dict.keys())

47

In [ ]:
for token in tokens:
    if not requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json'):
      print(f'API call for {token} failed')
      companies_bad[token] = companies_dict.get(token)
      companies_clean.pop(token)

API call for front failed
API call for boomandbucket failed


In [ ]:
companies_bad

{'front': None, 'boomandbucket': None}

In [ ]:
roles = {'developer','engineer', 'data', 'engineering', 'frontend', 'software', 'apprentice', 'analyst' , 'quality', 'apprenticeship', 'front-end', 'backend', 'back-end', 'jr.', 'jr' }
# Includes 'software' for titles that are just 'software enginer', etc.
levels = {'junior', 'entry-level', 'grad', 'graduate', 'data', 'apprentice', 'apprenticeship', 'software', 'qa', 'quality', 'test', 'entry', 'intern', 'i', '1', 'associate', 'co-op'}
# Optional, but helps exclude higher level postiions
exclude = {'senior', 'principal' , 'sr.', 'sr' 'ii', 'iii', 'director' }



In [ ]:
res=requests.get(f'https://jobs.lever.co/v0/postings/tegus?mode=json')
jobs=json.loads(res.text)
job = jobs[0]

job.keys()

dict_keys(['additionalPlain', 'additional', 'categories', 'createdAt', 'descriptionPlain', 'description', 'id', 'lists', 'text', 'country', 'workplaceType', 'hostedUrl', 'applyUrl'])

In [ ]:
job['categories']

{'commitment': 'Full-Time',
 'department': 'Operations',
 'location': 'Chicago, IL',
 'team': 'Operations'}

In [ ]:
print(datetime.datetime.fromtimestamp(job.get('createdAt')/1000.).isoformat())


2023-04-03T22:33:54.646000


In [ ]:
results = []

for token in tokens:
  res=requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json')
  if res:
    jobs=json.loads(res.text)

  company= companies_clean.get(token)

  for job in jobs:
    title = set(job.get('text').lower().split())

    if title.intersection(roles) and title.intersection(levels) and not title.intersection(exclude):
      job_info = {'title': job.get('text'), 'company' : company, 'description': job.get('descriptionPlain'),
                'link' : job.get('applyUrl'), 'remote' : True if job.get('workplaceType').lower() == 'remote' else False, 'location' : job.get('categories').get('location') ,  'date_posted': datetime.datetime.fromtimestamp(job.get('createdAt')/1000.).isoformat()} 
      
      results.append(job_info)



In [ ]:
df = pd.DataFrame()
df = df.from_records(results)
df

,title,company,description,link,remote,location,date_posted
0,Financial Intern - Data Governance,Tegus,Tegus is the leading market intelligence platf...,https://jobs.lever.co/tegus/c28f60ce-cc2d-432a...,False,"Vancouver, BC",2022-08-26T20:32:41.764000
1,Lead Software Engineer,Tegus,Tegus is the leading market intelligence platf...,https://jobs.lever.co/tegus/b05efb57-a5c1-4caa...,False,"Chicago, IL & Vancouver, BC",2023-01-26T23:18:06.940000
2,Software Engineer,Tegus,Tegus is the leading market intelligence platf...,https://jobs.lever.co/tegus/ce5a986d-061c-4937...,True,United States,2022-03-17T15:38:05.609000
3,Software Engineer Intern,Tegus,Tegus is the leading market intelligence platf...,https://jobs.lever.co/tegus/d6d1a369-4ef0-4808...,False,"Chicago, IL",2023-01-19T22:41:07.951000
4,Manager Data Science,Eargo,"We’re Eargo and we’re on a mission to disrupt,...",https://jobs.lever.co/eargo/77c4ab85-86ea-4c09...,True,United States,2023-01-09T21:32:54.772000
...,...,...,...,...,...,...,...
525,Cloud Data Architect (US-Remote Optional),Ascend.io,Ascend's mission is to enable every organizati...,https://jobs.lever.co/ascend.io/39e83e80-b0b5-...,True,Remote,2020-02-03T21:54:28.179000
526,Field Data Engineer (US-Remote Optional),Ascend.io,Ascend's mission is to enable every organizati...,https://jobs.lever.co/ascend.io/e4d284e9-2632-...,True,Remote,2021-04-28T01:26:35.178000
527,Software Engineer (US Remote Optional),Ascend.io,Ascend's mission is to enable every organizati...,https://jobs.lever.co/ascend.io/60a0ac22-7f65-...,True,Remote,2022-01-12T17:10:59.717000
528,"Software Engineer, Front-End",OtterTune,"Who We Are\nAt OtterTune, we make database opt...",https://jobs.lever.co/ottertune/7ce5d9b0-c54f-...,True,"Remote (US, Canada)",2020-10-14T04:58:04.105000


In [ ]:
df.sort_values(by='date_posted', ascending=False)

,title,company,description,link,remote,location,date_posted
484,Software Engineer - Mapping,Anduril Industries,"Anduril is a defense technology company, bring...",https://jobs.lever.co/anduril/ac3b5640-8025-4c...,False,"Seattle, WA",2023-04-04T18:54:42.085000
483,Software Engineer - Mapping,Anduril Industries,"Anduril is a defense technology company, bring...",https://jobs.lever.co/anduril/15970b65-4a9e-40...,False,"Costa Mesa, CA",2023-04-04T18:53:56.472000
463,Software Engineer - Motion Planning,Anduril Industries,"Anduril is a defense technology company, bring...",https://jobs.lever.co/anduril/25140a5f-ebe1-4d...,False,"Boston, MA",2023-04-04T18:52:11.672000
21,Associate Lead Activity Data - Link,Veeva,Veeva is a mission-driven organization that as...,https://jobs.lever.co/veeva/0b123483-f0eb-4bed...,True,Bulgaria - Sofia,2023-04-04T17:22:14.528000
22,Associate Lead Activity Data - Link,Veeva,Veeva is a mission-driven organization that as...,https://jobs.lever.co/veeva/297b3816-f09e-4f13...,True,Hungary - Budapest,2023-04-04T17:19:25.896000
...,...,...,...,...,...,...,...
315,Software Engineering Manager,Forward,Forward is on a bold mission to make high qual...,https://jobs.lever.co/goforward/83a37a5b-5ed8-...,False,"San Francisco, CA",2018-07-23T15:38:40.872000
295,Technical Program Manager - Software,Zoox,Zoox has embarked on a highly ambitious journe...,https://jobs.lever.co/zoox/b5b5bb0c-1f95-446e-...,False,"Foster City, CA",2016-05-17T17:37:06.372000
221,Senior/Staff Software Engineer - Machine Learning,Zoox,"At Zoox, you will collaborate with a team of w...",https://jobs.lever.co/zoox/81f7fbe7-4fd3-4814-...,False,"Foster City, CA",2016-02-12T01:45:48.453000
256,Software Development Engineer in Test,Zoox,"Autonomous vehicles have some of the largest, ...",https://jobs.lever.co/zoox/206cbbe7-f2ef-40f1-...,False,"Foster City, CA",2016-02-11T22:06:53.811000
